In [ ]:
import logging
import joblib
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import os
import nest_asyncio
import asyncio
from collections import defaultdict

nest_asyncio.apply()  # ✅ Fix for Google Colab event loop error

# Load Telegram Bot Token
BOT_TOKEN = ""

# Load Models & Preprocessing Tools
text_model = load_model(".../text_model1.h5")
tokenizer = joblib.load(".../1vectorizer.pkl")

image_model = load_model(".../image_model.keras")

metadata_model = joblib.load(".../metadata_fraud_model.pkl")
tfidf_vectorizer = joblib.load(".../tfidf_vectorizer.pkl")
scaler = joblib.load(".../scaler.pkl")

# Recompile models to avoid warnings
text_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
image_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Set Up Logging
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

# Fraud Message Storage (Store 2 fraud messages per user)
user_fraud_messages = defaultdict(list)

# Admin Details
your_telegram_id =   # Only send reports to you

# Detect Fraud in Text
def detect_text_fraud(user_id, message):
    sequence = tokenizer.texts_to_sequences([message])
    padded = pad_sequences(sequence, maxlen=200)
    fraud_probability = text_model.predict(padded)[0][0]

    if fraud_probability > 0.7:
        risk_level = "🚨 Very High Risk"
    elif fraud_probability > 0.5:
        risk_level = "⚠️ High Risk"
    elif fraud_probability > 0.3:
        risk_level = "🟡 Medium Risk"
    else:
        risk_level = "✅ Low Risk"

    # Store fraud messages for each user
    if fraud_probability > 0.5:
        if len(user_fraud_messages[user_id]) >= 2:
            user_fraud_messages[user_id].pop(0)  # Keep only the last 2 messages
        user_fraud_messages[user_id].append(f"🔹 {message} ({risk_level})")

    return fraud_probability, risk_level

# Generate Fraud Report
async def send_fraud_report(application, user_id, username):
    if len(user_fraud_messages[user_id]) == 1:  # ✅ Send report after 2 fraud detections
        report = (
            f"📢 *Fraud Alert Detected!*\n"
            f"👤 *User:* {username or 'Unknown'} (ID: {user_id})\n"
            f"🚨 *Risk Level:* High\n\n"
            f"📌 *Fraudulent Messages:*\n"
            + "\n".join(user_fraud_messages[user_id])
        )

        # ✅ Send report only to Admin (You)
        await application.bot.send_message(your_telegram_id, report, parse_mode="Markdown")

        # ✅ Clear stored messages after reporting
        user_fraud_messages[user_id] = []

# Handle Incoming Messages (Text) - No Reply in Group
async def handle_message(update: Update, context: CallbackContext):
    user_message = update.message.text
    user_id = update.message.from_user.id
    username = update.message.from_user.username

    fraud_probability, risk_level = detect_text_fraud(user_id, user_message)

    # Send fraud report if necessary
    await send_fraud_report(context.application, user_id, username)

# Detect Fraud in Images
def detect_image_fraud(user_id, image_path):
    """Loads an image, preprocesses it, and predicts fraud probability."""
    expected_size = image_model.input_shape[1:3]  # ✅ Get model's required input size

    img = image.load_img(image_path, target_size=expected_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    fraud_probability = image_model.predict(img_array)[0][0]

    if fraud_probability > 0.7:
        risk_level = "🚨 Very High Risk"
    elif fraud_probability > 0.5:
        risk_level = "⚠️ High Risk"
    elif fraud_probability > 0.3:
        risk_level = "🟡 Medium Risk"
    else:
        risk_level = "✅ Low Risk"

    # Store fraud images for user
    if fraud_probability > 0.5:
        user_fraud_messages[user_id].append(f"📸 Fraudulent Image Detected ({risk_level})")

    return fraud_probability, risk_level

# Handle Incoming Images - No Reply in Group
async def handle_photo(update: Update, context: CallbackContext):
    photo_file = await update.message.photo[-1].get_file()
    image_path = "downloaded_image.jpg"
    user_id = update.message.from_user.id
    username = update.message.from_user.username

    try:
        await photo_file.download_to_drive(image_path)  # ✅ Corrected function
        fraud_probability, risk_level = detect_image_fraud(user_id, image_path)

        # ✅ Send fraud report if necessary
        await send_fraud_report(context.application, user_id, username)

    except Exception as e:
        logging.error(f"❌ Error processing image: {str(e)}")

# Handle Videos (Extract First Frame) - No Reply in Group
async def handle_video(update: Update, context: CallbackContext):
    video_file = await update.message.video.get_file()
    video_path = "downloaded_video.mp4"
    user_id = update.message.from_user.id
    username = update.message.from_user.username

    try:
        await video_file.download_to_drive(video_path)  # ✅ Corrected function
        cap = cv2.VideoCapture(video_path)
        success, frame = cap.read()

        if success:
            frame_path = "video_frame.jpg"
            cv2.imwrite(frame_path, frame)
            fraud_probability, risk_level = detect_image_fraud(user_id, frame_path)

            # ✅ Send fraud report if necessary
            await send_fraud_report(context.application, user_id, username)

    except Exception as e:
        logging.error(f"❌ Error processing video: {str(e)}")

# Handle /start Command (Only Works in Private Chat)

# Set Up Telegram Bot Handlers

# Run the Bot
